In [1]:
import datetime
import pytz

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplcyberpunk

from pathlib import Path
import pandas as pd
import numpy as np
from astropy.io import fits
import astropy.wcs as fitswcs
import astropy.units as u
from astropy.coordinates import EarthLocation, SkyCoord, AltAz, get_moon, Angle
from astropy.time import Time
from astropy.table import Table
import astroplan
from specutils.spectra.spectrum1d import Spectrum1D
from astropy.visualization import (MinMaxInterval, SqrtStretch, ZScaleInterval,
                                   ImageNormalize, quantity_support)
from specutils.manipulation import FluxConservingResampler

from sklearn.decomposition import PCA

_ = quantity_support()
plt.style.use("cyberpunk")
SMALL_SIZE = 18
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('font', family='Impact')
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [12]:
convert_22mag = 143.1685 * 10 ** (-0.4 * 22)

In [15]:
df = pd.read_html("viirs_2021.html")[0]
df['zenith_ratio'] = 100 * df['Zenith radiance'] / convert_22mag
df

,Site,Type,Longitude,Latitude,Altitude,Zenith radiance,Average radiance (all sky),Average radiance at 30,Average radiance below 10,Horizontal irradiance,zenith_ratio
0,Maui A.F. Optical Station,Observatory,-156.25700,20.70820,3036,1.170000e-08,4.190000e-08,2.840000e-08,1.640000e-07,8.350000e-08,5.156302
1,Mauna Kea,Observatory,-154.47253,20.82464,4160,1.730000e-09,9.250000e-09,4.960000e-09,4.400000e-08,1.560000e-08,0.762428
2,Lick,Observatory,-121.63732,37.34285,1260,3.060000e-07,1.290000e-06,8.930000e-07,3.880000e-06,2.390000e-06,134.857142
3,Palomar,Observatory,-116.86500,33.35660,1712,1.340000e-07,7.990000e-07,4.650000e-07,2.700000e-06,1.300000e-06,59.055088
4,San Pedro Martir,Observatory,-115.46370,31.04410,2795,2.140000e-09,1.660000e-08,6.440000e-09,1.310000e-07,2.100000e-08,0.943119
5,Kitt Peak,Observatory,-111.59991,31.96394,2050,1.340000e-08,1.330000e-07,6.860000e-08,4.290000e-07,2.000000e-07,5.905509
6,Lowell-DCT,Observatory,-111.42200,34.74460,2353,5.830000e-09,7.510000e-08,3.550000e-08,2.490000e-07,1.060000e-07,2.569337
7,MMTO/FLWO,Observatory,-110.88507,31.68892,2580,2.830000e-08,2.590000e-07,1.440000e-07,8.350000e-07,4.030000e-07,12.472082
8,Mount Graham,Observatory,-109.88920,32.70128,3191,1.360000e-08,1.100000e-07,5.910000e-08,3.440000e-07,1.720000e-07,5.993651
9,Starfire Optical Range,Observatory,-106.46349,34.96415,1872,2.270000e-07,2.030000e-06,1.260000e-06,6.620000e-06,3.280000e-06,100.041082
